In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 22.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orbax-checkpoint 0.3.3 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.
pydantic 2.1.1 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.4.0 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [ ]:
!pip install pymongo
!pip install pandas

In [ ]:
import pymongo
from pymongo import MongoClient
import pandas as pd

In [ ]:
def getDataFiles():
  connection_string = <Add your database url>
  client = MongoClient(connection_string)
  db = client['flipkartDB']
  collections = ['products','transactions','interactions',"users"]
  for name in collections:
    collection = db[name]
    query={}
    cursor = collection.find(query)
    df = pd.DataFrame(list(cursor))
    df.to_csv(f"/content/{name}.csv")


In [ ]:
getDataFiles()

In [ ]:
import os
import pprint
import tempfile

# from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import pandas as pd

###Data Loading and preprocessing

In [ ]:
interactions = pd.read_csv("/content/flipkartDB.interactions.csv",dtype={"userId":str})
users = pd.read_csv("/content/flipkartDB.users.csv")
products = pd.read_csv("/content/flipkartDB.products.csv")

<ipython-input-88-40ef383028d2>:2: DtypeWarning: Columns (13,14,21,22,48,52,53,54,55,56,57,58,59,85,89,90,91,92,93,94,95,96,159,163,164,165,166,167,168,169,170,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv("/content/flipkartDB.users.csv")


In [ ]:
interactions = interactions.head(10000)
products = products.head(10000)

In [ ]:
interactions = interactions[["userId","productId","point"]]
products.rename(columns={"_id":"productId"},inplace=True)
users.rename(columns={"_id":"userId"},inplace=True)
# movies = movies[["name","productId"]]
products['name']=products['name'] + ' ' + products['company'] + ' '+ products["description"]

merge_df = interactions.merge(products, on="productId", how="left")
merge_df = merge_df[["userId","productId","name","point"]]
# merge_df.rename(columns={"name":"name", "userId":"userId"}, inplace=True)
# product = pd.DataFrame(products["productId"].unique(), columns=["productId","name"])
products_tf = tf.data.Dataset.from_tensor_slices(dict(products))
products_tf = tf.data.Dataset.prefetch(products_tf,buffer_size=tf.data.AUTOTUNE)

In [ ]:
interactions_tf = tf.data.Dataset.prefetch(tf.data.Dataset.from_tensor_slices(dict(merge_df)), buffer_size=tf.data.AUTOTUNE)
# Converting ratings dataset to map dataset
interactions_map = interactions_tf.map( lambda x: {
"userId": x["userId"],
"productId": x["productId"],
"name":x["name"],
"point": x["point"]})
# Creating a map_dataset of movie data
product_map = products_tf.map(lambda x:x["name"])
product_dataset = products_tf.map(lambda x:{
    "productId":x["productId"],
    "name":x["name"]
})
product_ids = products_tf.map(lambda x:x["productId"])
# Generating unique values:
unique_user_ids = users["userId"].unique()
unique_products = products["productId"].unique()

###defining Models

In [ ]:
class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # self.use_timestamp = use_timestamp
        self.user_embeddings = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids)+1, 32)
            ])
    def call(self, inputs):
        return self.user_embeddings(inputs["userId"])

In [ ]:
class ProductModel(tf.keras.Model):
    def __init__(self):

        super().__init__()
        max_tokens = 10_000
        self.product_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_products, mask_token=None),
            tf.keras.layers.Embedding(len(unique_products)+1, 32)
            ])
        self.text_vectorizer =  tf.keras.layers.TextVectorization(max_tokens=max_tokens)
        self.product_names_context_embedding = tf.keras.Sequential([
            self.text_vectorizer,
            tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
            tf.keras.layers.GlobalAveragePooling1D()
            ])
        self.text_vectorizer.adapt(product_map)

    def call(self, inputs):
        return tf.concat([
            self.product_embedding(inputs["productId"]),
            self.product_names_context_embedding(inputs["name"])],
              axis=1)


In [ ]:
class ProductRecommendModel(tfrs.models.Model):
    def __init__(self, rating_weight, retrieval_weight):
        super().__init__()
        embedding_dimension = 32
        self.query_model = tf.keras.Sequential([
            UserModel(),
            tf.keras.layers.Dense(embedding_dimension)
          ])
        self.candidate_model = tf.keras.Sequential([
            ProductModel(),
            tf.keras.layers.Dense(embedding_dimension)
          ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1)
          ])
        self.retrieval_task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=product_dataset.batch(128).map(self.candidate_model)))
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()
          ])
       # The loss weights.
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight
    def call(self, features) -> tf.Tensor:
        user_embeddings = self.query_model({"userId": features["userId"]})
        product_embeddings = self.candidate_model({"productId":features["productId"],"name":features["name"]})
        return (user_embeddings, product_embeddings, self.rating_model(tf.concat([user_embeddings, product_embeddings],axis=1)))
    def compute_loss(self, features, training=False) -> tf.Tensor:

        ratings = features.pop("point")
        user_embeddings, product_embeddings, rating_predictions = self(features)
        # We compute the loss for each task.
        rating_loss = self.rating_task(labels=ratings, predictions=rating_predictions)
        retrieval_loss = self.retrieval_task(user_embeddings, product_embeddings)
        # And combine them using the loss weights.
        return (self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss)

In [ ]:
model = ProductRecommendModel(2, 1)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.5),metrics=['accuracy'])
# model.fit(train, epochs=10)
# <_MapDataset element_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None)>
# here

In [ ]:
train = interactions_map.take(8000)
test = interactions_map.skip(8000).take(2000)
cached_train = train.batch(128)
cached_test = test.batch(128)

In [ ]:
model.fit(cached_train,epochs=1)

63/63 [==============================] - 40s 612ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0209 - factorized_top_k/top_5_categorical_accuracy: 0.0233 - factorized_top_k/top_10_categorical_accuracy: 0.0249 - factorized_top_k/top_50_categorical_accuracy: 0.0342 - factorized_top_k/top_100_categorical_accuracy: 0.0415 - root_mean_squared_error: 592.3730 - loss: 692227.7042 - regularization_loss: 0.0000e+00 - total_loss: 692227.7042


In [ ]:
model.evaluate(cached_test,return_dict=True)

16/16 [==============================] - 11s 687ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 5.0000e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0010 - factorized_top_k/top_50_categorical_accuracy: 0.0045 - factorized_top_k/top_100_categorical_accuracy: 0.0090 - root_mean_squared_error: 237.2788 - loss: 115072.0450 - regularization_loss: 0.0000e+00 - total_loss: 115072.0450


{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0005000000237487257,
 'factorized_top_k/top_10_categorical_accuracy': 0.0010000000474974513,
 'factorized_top_k/top_50_categorical_accuracy': 0.0044999998062849045,
 'factorized_top_k/top_100_categorical_accuracy': 0.008999999612569809,
 'root_mean_squared_error': 237.27883911132812,
 'loss': 126557.9296875,
 'regularization_loss': 0,
 'total_loss': 126557.9296875}

In [ ]:
scann = tfrs.layers.factorized_top_k.ScaNN(
        model.query_model,
        num_leaves=1000,
        num_leaves_to_search=100,
        num_reordering_candidates=100,
        k=15)
candidate_embeddings = product_dataset.batch(128).map(lambda x:model.candidate_model(x))
# for i in lets_products_embeddings.take(5):
#   print(i)
scann.index_from_dataset(tf.data.Dataset.zip((product_ids.batch(128),candidate_embeddings)))

In [ ]:
def decode_titles(titles):
  def decode_bytes(data):
    return data.decode('utf-8')
# res = np.vectorize(decode)(titles.numpy())
  data=[]
  for i in titles:
    data = np.vectorize(decode_bytes)(i.numpy())
  return data


In [ ]:
def getRecommendations(userId):
  _,titles = scann({"userId":tf.constant([userId])})
  prods = decode_titles(titles)
  res = {"productId":[],"point":[]}
  test_data={"userId":tf.constant([userId])}
  for data in prods:
    res["productId"].append(data)
    row=products.loc[products["productId"]==data]
    row=row["name"]
    test_data["productId"]=np.array([data])
    test_data["name"]=np.array([row])
    _, _, rating = model(test_data)
    res["point"].append(int(rating[0][0]))
  # for data in prods:
  #   test_data["productId"]=np.array([data])
  df_res = pd.DataFrame(res)
  # df_res=df_res.sort_values('point', ascending=False)
  # recommends = np.array(df_res["productId"])
  # print(df_res)
  return df_res

In [ ]:
def getUserRecommendations(userId):
  user_res_df=getRecommendations(userId)
  user_res_df=user_res_df.sort_values('point', ascending=False)
  recommends = np.array(user_res_df["productId"])
  return list(recommends)

In [ ]:
def getLocationRecommendations(location):
  users_local = users.loc[users["location"]==location].head(10)["userId"]
  recommends = pd.DataFrame()
  for user in users_local:
    local = getRecommendations(user)
    recommends = recommends.append(local)
  recommends = recommends.sort_values('point',ascending=False)
  recommends = list(recommends['productId'])
  return recommends

In [ ]:
def getAgeRecommendations(age):
  users_local = users.loc[users["age"]==age].head(10)["userId"]
  recommends = pd.DataFrame()
  for user in users_local:
    local = getRecommendations(user)
    recommends = recommends.append(local)
  recommends = recommends.sort_values('point',ascending=False)
  recommends = list(recommends['productId'])
  return recommends

In [ ]:
getLocationRecommendations("Gujarat")

In [ ]:
getUserRecommendations("64df99c726ae34e9b5e8f4fe")

In [ ]:
getAgeRecommendations("Child")

In [ ]:
!pip install -q colabcode
!pip install -q fastapi

In [ ]:
from colabcode import ColabCode
from fastapi import FastAPI
cc = ColabCode(port=12000, code=False)

In [ ]:
app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

# @app.on_event("startup")
# def load_model():
#     global model
#     model = pickle.load(open("model_gb.pkl", "rb"))

@app.post("/api/home")
def get_predictions(data:dict):
    try:
        recommends = getUserRecommendations(data["userId"])
        print(type(recommends))
        return recommends
    except Exception as e:
        print(e)
        return {"error": e}

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://b18d-34-29-10-228.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [924]
INFO:uvicorn.error:Started server process [924]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


<class 'list'>
INFO:     117.239.204.225:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.204.225:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.204.225:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.210.100:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.210.100:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.210.100:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.210.100:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.210.100:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.204.225:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.204.225:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.204.225:0 - "POST /api/home HTTP/1.1" 200 OK
<class 'list'>
INFO:     117.239.204.225:0 - "POST /api/home HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [924]
INFO:uvicorn.error:Finished server process [924]
